<a href="https://colab.research.google.com/github/rociolopeth/deccan-traps-gee/blob/main/deccan_traps_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>